<a href="https://colab.research.google.com/github/monksoo/restaurant-sentient-analysis/blob/main/junior_chicken_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U requests

import requests
import json

api_key = 'vWIUxpfwQGIcXvFDJnNOMq7rUHHVZvBYcVgdAA-0gFtb_BNZk35ctOBzyf8AT_UOPubJj_CP2b6h-n9uXchwRi4-WxLhz8hWlMfNI2VB34ta91e-ruWWC7hCxVAaZHYx'
headers = {'Authorization': f'Bearer {api_key}'}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


In [2]:
# Search parameters
term = 'restaurants'
location = 'Canada'

# API endpoint
url = f'https://api.yelp.com/v3/businesses/search?term={term}&location={location}'

# Make a GET request to the API
response = requests.get(url, headers=headers)

# Convert response to JSON
data = json.loads(response.text)

# Extract review data from the response
reviews = []
for business in data['businesses']:
    business_id = business['id']
    reviews_url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
    response = requests.get(reviews_url, headers=headers)
    review_data = json.loads(response.text)
    for review in review_data['reviews']:
        reviews.append({'text': review['text'], 'rating': review['rating']})

In [3]:
# Load NLTK resources
!pip install -U nltk

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load reviews data from list
import pandas as pd

reviews_df = pd.DataFrame(reviews)

# Remove punctuation and special characters
def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

reviews_df['text'] = reviews_df['text'].apply(lambda x: remove_punct(x))

# Convert text to lowercase
reviews_df['text'] = reviews_df['text'].apply(lambda x: x.lower())

# Remove stop words
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)

reviews_df['text'] = reviews_df['text'].apply(lambda x: remove_stopwords(x))

# Stem words
stemmer = PorterStemmer()

def stem_words(text):
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

reviews_df['text'] = reviews_df['text'].apply(lambda x: stem_words(x))

# Save cleaned data to a CSV file
reviews_df.to_csv('reviews.csv', index=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Load cleaned data from CSV file
reviews_df = pd.read_csv('reviews.csv')

In [5]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews_df['text'], reviews_df['rating'], test_size=0.2, random_state=42)

In [6]:
# Vectorize text data
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [7]:
# Train a machine learning model
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train_vec, y_train)

LinearSVC()

In [8]:
# Evaluate the model on the testing data
from sklearn.metrics import accuracy_score

y_pred = svm.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy:.2f}')

Test accuracy: 0.58


In [9]:
# Classify a new review
new_review = 'This restaurant has great food and service!'
new_review_vec = vectorizer.transform([new_review])
new_review_rating = svm.predict(new_review_vec)[0]
print(f'The predicted rating for the new review is {new_review_rating}')

The predicted rating for the new review is 5


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import pickle

# Define the machine learning model as a scikit-learn Pipeline object
model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression())
])

# Train the model on some training data
X_train = ['This restaurant is amazing!', 'The food was terrible.']
y_train = ['positive', 'negative']
model.fit(X_train, y_train)

# Save the model to a file using the pickle module
with open('juniorchicken', 'wb') as f:
    pickle.dump(model, f)


In [11]:
import pickle

# Assume that you have trained and evaluated your model and you have
# stored it in a variable named 'model'

# Save the model to a file named 'model.pkl'
with open('juniorchicken', 'wb') as f:
    pickle.dump(model, f)
